1\. Write a function that converts number representation (bin<->dec<->hex)

In [38]:
trans={'A':10, 'B':11, 'C':12, 'D':13, 'E':14, 'F':15,'1':1,'2':2, '3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'0':0}
invtrans = {v: k for k, v in trans.items()}



def rd(num,base):
    a=str(num)
    r=0
    for i in range(0,len(a)):
        r=r+trans[a[i]]*(base**(len(a)-1-i))
    return r
             
def rapp(num,base):
    r=""
    while num>=base:
        r=str(invtrans[num%base])+r
        num=num//base
    r=str(invtrans[num%base])+r
    num=num//base
    return r
    
    
    
def b12(n,b1,b2):
    return rapp(int(rd(n,b1)),b2)
    
    
    
    
    
a="1001"
print(rd(a,2))
print(rapp(int(rd(a,2)),2))

print(b12(a,2,4))

9
1001
21


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [41]:
#def t(s):
#    a=int(s)
#    sign=a//10**31
#    exp=(a//10**23)%(10**31)
#    man=(a)%(10**23)
#    return (((-1)**(sign))*((int(1+(man/(10**23)),2))))**(int(exp,2)-127)
    
def t(s):
    
    


TypeError: int() can't convert non-string with explicit base

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 
